In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head()

In [ ]:
df.iloc[0,:]

In [ ]:
df.info()

In [ ]:
df[df['TotalCharges'] == " "]

11 from 7043 entries. There is two options drop them It is less than 0.2% or replace missing values by mean or use more advanced technique.

In [ ]:
df = df.drop( index = df[df['TotalCharges'] == " "].index)
df.reset_index(inplace = False)
df



In [ ]:
df["TotalCharges"] =pd.to_numeric(df["TotalCharges"])
df.info()

In [ ]:
df.describe()

basic view on numeric features distribution

In [ ]:
df.Churn.unique()

In [ ]:
df.Churn.value_counts()

distribution of our dependent variable

In [ ]:
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected = True)

labels = ['No','Yes']
values = df.Churn.value_counts()

fig = go.Figure(data=[go.Pie(labels=labels, values=values
                            )])
fig.update_layout(title_text='Churn')
fig.show()



In [ ]:
col = df.columns[1:-1] # first columns is customer ID and last is Churn
col

distribution of all variables depending on Churn

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import copy

a = 10  # number of rows
b = 2  # number of columns
c = 1  # initialize plot counter

fig = plt.figure(figsize=(20,80))

for i in range(len(col)):
    xx = copy.deepcopy(col)
    plt.subplot(a, b, c)
    plt.title('{}'.format(i))
    plt.xlabel(xx[i])
    sns.countplot(x=xx[i], hue="Churn", data=df)
    c = c + 1

plt.show()

In [ ]:
sns.boxplot(y=df.tenure, x=df.Churn)

In [ ]:
sns.boxplot(y=df.MonthlyCharges, x=df.Churn)

In [ ]:
sns.boxplot(y=df.TotalCharges, x=df.Churn)

cathegoriacal variables to numeric. 
2 unique elemnets --> labelencoder, 
more than 2 elements -->dummy encoder

In [ ]:
obj_df = df.select_dtypes(include=['object']).copy()
obj_df

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)


In [ ]:
for i in df.columns:
    print(df[i].unique())

In [ ]:
a = ["gender","Partner", "Dependents", "PhoneService", "PaperlessBilling", "Churn"]
encoded_df = MultiColumnLabelEncoder(columns = a).fit_transform(df)
encoded_df

In [ ]:
b = obj_df.drop(["Churn","customerID", "gender","Partner", "Dependents", "PhoneService", "PaperlessBilling"], axis =1)
b = b.columns
b =pd.Series(b)

final_df = pd.get_dummies(columns=b ,data = encoded_df)
final_df.head()

In [ ]:
final_df.info()

In [ ]:
final_df.corr()

In [ ]:
final_df.corr()['Churn'].sort_values(ascending=False)

In [ ]:
X = final_df.drop(['Churn', 'customerID'],axis=1)
Y = final_df['Churn']

from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=0)
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

spliting data set

In [ ]:
#Logistic regression
from sklearn.linear_model import LogisticRegression
#from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
model = LogisticRegression(max_iter = 200)

model.fit(X_train,Y_train)

In [ ]:
prediction=model.predict(X_test)
prediction[0:10]

In [ ]:
Y_test[0:10]

In [ ]:
print(classification_report(Y_test,prediction))

In [ ]:
print("Model Accuracy:",model.score(X_test,Y_test))

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot

# predict probabilities
lr_probs = model.predict_proba(X_test)
# keep probabilities for the positive outcome only
lr_probs = lr_probs[:, 1]
# generate a no skill prediction
ns_probs = [0 for _ in range(len(Y_test))]
# calculate scores
ns_auc = roc_auc_score(Y_test, ns_probs)
lr_auc = roc_auc_score(Y_test, lr_probs)
# summarize scores
print('Random choose: ROC AUC=%.3f' % (ns_auc))
print('Logistic: ROC AUC=%.3f' % (lr_auc))

In [ ]:
# calculate roc curves
ns_fpr, ns_tpr, _ = roc_curve(Y_test, ns_probs)
lr_fpr, lr_tpr, _ = roc_curve(Y_test, lr_probs)
# plot the roc curve for the model
pyplot.plot(ns_fpr, ns_tpr, linestyle='--', label='Random choose')
pyplot.plot(lr_fpr, lr_tpr, marker='.', label='Logistic')
# axis labels
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
pyplot.title('ROC curve')
plt.grid(True)
# show the legend
pyplot.legend()
# show the plot
pyplot.show()

In [ ]:
# random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
model_rf = RandomForestClassifier()
parameters = {'max_depth': [2, 5,10], 'n_estimators':[10,100,1000]}

clf = GridSearchCV(model_rf,parameters, scoring = 'precision', verbose = 10, n_jobs = -1 )



In [ ]:
clf.fit(X_train,Y_train)

In [ ]:
pred_rfc= clf.predict(X_test)
pred_rfc[:10]

In [ ]:
Y_test[:10]

In [ ]:
print(classification_report(Y_test,pred_rfc))

In [ ]:
print("Model Accuracy:",clf.score(X_test,Y_test))

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot

# predict probabilities
lr_probs = clf.predict_proba(X_test)
# keep probabilities for the positive outcome only
lr_probs = lr_probs[:, 1]
# generate a no skill prediction
ns_probs = [0 for _ in range(len(Y_test))]
# calculate scores
ns_auc = roc_auc_score(Y_test, ns_probs)
lr_auc = roc_auc_score(Y_test, lr_probs)
# summarize scores
print('Random choose: ROC AUC=%.3f' % (ns_auc))
print('RandomForest: ROC AUC=%.3f' % (lr_auc))

In [ ]:
# calculate roc curves
ns_fpr, ns_tpr, _ = roc_curve(Y_test, ns_probs)
lr_fpr, lr_tpr, _ = roc_curve(Y_test, lr_probs)
# plot the roc curve for the model
pyplot.plot(ns_fpr, ns_tpr, linestyle='--', label='Random choose')
pyplot.plot(lr_fpr, lr_tpr, marker='.', label='RandomForest')
# axis labels
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
pyplot.title('ROC curve')
plt.grid(True)
# show the legend
pyplot.legend()
# show the plot
pyplot.show()